Copyright (c) Microsoft Corporation.

Licensed under the MIT License.

# Text2SQL with AutoGen & Azure OpenAI

This notebook demonstrates how the AutoGen Agents can be integrated with Azure OpenAI to answer questions from the database based on the schemas provided. 

A multi-shot approach is used for SQL generation for more reliable results and reduced token usage. More details can be found in the README.md.

### Dependencies

To install dependencies for this demo:

`uv sync --package autogen_text_2_sql`

`uv add --editable text_2_sql_core`

In [1]:
# This is only needed for this notebook to work
import sys
from pathlib import Path

# Add the parent directory of `src` to the path
sys.path.append(str(Path.cwd() / "src"))

In [2]:
import dotenv
import logging
from autogen_agentchat.ui import Console
from autogen_text_2_sql.autogen_text_2_sql import AutoGenText2Sql

In [3]:
logging.basicConfig(level=logging.ERROR)

In [ ]:
dotenv.load_dotenv()

## Bot setup

In [5]:
agentic_text_2_sql = AutoGenText2Sql(engine_specific_rules="", use_case="Analysing sales data")

### Test Simple Query

In [ ]:
# Process the question and get the stream
stream = agentic_text_2_sql.process_question(
    task="What total number of orders in June 2008?",
    parameters={"date": "2008-06-01"}
)

# Display the streaming results
await Console(stream)

### Test 1: Complex Query with Independent Sub-queries

This test demonstrates parallel processing of independent sub-queries. Each metric can be calculated independently and combined in the final result.

In [ ]:
# reset
agentic_text_2_sql = AutoGenText2Sql(engine_specific_rules="", use_case="Analysing sales data")

# Process multiple independent metrics in parallel
stream = agentic_text_2_sql.process_question(
    task="""Compare the following metrics for 2008:
    1. Total number of orders
    2. Average order value
    3. Number of unique customers
    4. Top selling product category""",
    parameters={"year": "2008"}
)

# Display the streaming results
await Console(stream)

### Test 2: Query with Dependent Sub-queries

This test demonstrates sequential processing where later queries depend on earlier results. The system will process these queries in the correct order based on their dependencies.

In [ ]:
# reset
agentic_text_2_sql = AutoGenText2Sql(engine_specific_rules="", use_case="Analysing sales data")

# Process dependent queries sequentially
stream = agentic_text_2_sql.process_question(
    task="""Find the top customer by total order value in 2008, then:
    1. List all their orders
    2. Calculate their average order value
    3. Show what product categories they prefer""",
    parameters={"year": "2008"}
)

# Display the streaming results
await Console(stream)

### Test 3: Mixed Independent and Dependent Queries

This test combines both parallel and sequential processing. After identifying the best-selling category, multiple analysis queries can run in parallel.

In [ ]:
# reset
agentic_text_2_sql = AutoGenText2Sql(engine_specific_rules="", use_case="Analysing sales data")

# Process mixed parallel and sequential queries
stream = agentic_text_2_sql.process_question(
    task="""Analyze our best-selling product category:
    1. First identify the category with highest total sales
    2. Then simultaneously calculate:
       - Monthly sales trend for this category
       - Top 5 customers for this category""",
    parameters={"date_range": "all_time"}
)

# Display the streaming results
await Console(stream)